RAIN - Real & Artificial Intelligence for Neuroscience

## Create models
- This notebook will create and train Artificial Neural Networks to identify exploration using rodent and target position along with manually labeled data.

#### Requirements:

- A set of position files
- Labeled data for those position files (to train the model)

or

- Access to the example file **colabels.csv**, where we can find:
    - Position and labels for representative exploration events
    - Manual labels from 5 viewers (so far)

---
#### Load the necessary modules

In [1]:
from pathlib import Path
import rainstorm.modeling as rst

rainstorm.modeling successfully imported. GPU devices detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


---
#### 1. State your models project path
`base` : The path to the downloaded repository.

`models_folder` : The path to the folder containing the files you'll use to create the models.

In [2]:
base = Path(r'c:\Users\dhers\Desktop\Rainstorm')
models_folder = base / r'examples\models'

---
#### Create the colabels file

The Colabels file is a csv file that contains both mice positions, exploration target positions, and one or more sets of labels for the behavior you want to analyze.

If you want to train the models using your own colabels file, you can create it using the create_colabels function below. 

All you need is:
- A folder containing the positions of the mice
- A folder for each labeler, containing the labels for the behavior you want to analyze
- A list of the targets (stationary points) present on your videos


```python
path = r'path/to/colabels_folder' # The path to the directory containing the positions folder and labelers folders
labelers = ['labeler_A', 'labeler_B', 'etc']
targets = ['tgt_1', 'tgt_2', 'etc']

rst.create_colabels(path, labelers, targets)
```

##### There is a Colabels file available in the models folder which contains positions and labels for mice on a novel object recognition task. If you want to analyze the demo data, train the models using that colabels file.

---
#### 2.  Create the modeling.yaml file

The modeling.yaml file is a configuration file that contains all the parameters needed to create and train the models. It will be located in the models folder.

In [3]:
modeling = rst.create_modeling(models_folder)

modeling.yaml already exists: c:\Users\dhers\Desktop\Rainstorm\examples\models\modeling.yaml
Skipping creation.



It contains the following parameters:

`path` : Path to the models folder

`colabels` : The colabels file is used to store and organize positions and labels for model training
- colabels_path: Path to the colabels folder
- labelers: List of labelers on the colabels file (as found in the columns)
- target: Name of the target on the colabels file

`focus_distance`: Window of frames to consider around an exploration event

`bodyparts`: List of bodyparts used to train the model

`split`: Parameters for splitting the data into training, validation, and testing sets
- validation: Percentage of the data to use for validation
- test: Percentage of the data to use for testing

`RNN`: Set up the Recurrent Neural Network
- width: Defines the shape of the wide model
  - past : Number of past frames to include
  - future : Number of future frames to include
  - broad : Broaden the window by skipping some frames as we stray further from the present
- units: Number of neurons on each layer
- batch_size: Number of training samples the model processes before updating its weights
- epochs: Each epoch is a complete pass through the entire training dataset
- lr: Learning rate


---
#### 3. Before training a model, we need to prepare our training data
- First, we load the dataset from the colabels file and create one 'labels' column out of all the labelers.
- Next (optional, but recommended) we can erase the rows of the dataset that are too far away from exploration events.
- Finally, we split the dataset into training, testing and validation subsets.

In [4]:
# Prepare the data
dataset = rst.prepare_data(modeling)

# Focus on the rows near exploratory behaviour
dataset = rst.focus(modeling, dataset)

# Split the data
model_dict = rst.split_tr_ts_val(modeling, dataset)

# Save the split
rst.save_split(modeling, model_dict)

Focused around 'labels' events (10231 found)
Rows reduced: 167012 -> 38161
📊 Splitting data into training, validation, and test sets...
Training set:    27281 samples
Validation set:  5489 samples
Testing set:     5391 samples
Total samples:   38161
💾 Split data saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\splits\split_2025-06-12.h5


WindowsPath('c:/Users/dhers/Desktop/Rainstorm/examples/models/splits/split_2025-06-12.h5')

---
If you later want to load a previous split, run:

```python
saved_split = models_folder / 'splits/split_{example_date}.h5' # Select the split you want to rescue
model_dict = rst.load_split(saved_split)
```

---
We can see on the testing data that the exploratory events happen when the nose gets close to the object

In [5]:
rst.plot_example_data(model_dict['X_ts'], model_dict['y_ts'])

---
#### 4. With the training data ready, we can use TensorFlow to design your very first model
- It will look at the positions of one frame at a time, and try to decide if the mouse is exploring.
- If the decision is correct the architecture will be reinforced, else it will be corrected according to the learning rate.
- We will train it for some epochs (cycles through the whole dataset) and plot how the accuracy and loss evolve.
- Also, we will be validating the training using the validation split, which contains frames that were not used for training.

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
import datetime
time = datetime.datetime.now()

# Build a simple neural network
model_simple = tf.keras.Sequential([
    
    # Input layer
    Input(shape=(model_dict['X_tr'].shape[1],)), 

    # Hidden layers
    Dense(32, activation='relu'),
    Dense(24, activation='relu'),
    Dense(16, activation='relu'),
    Dense(12, activation='relu'),
    Dense(8, activation='relu'),
    
    # Output layer
    Dense(1, activation='sigmoid')
])

# Compile the model
model_simple.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), 
                   loss='binary_crossentropy', metrics=['accuracy'])

model_simple.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                416       
                                                                 
 dense_1 (Dense)             (None, 24)                792       
                                                                 
 dense_2 (Dense)             (None, 16)                400       
                                                                 
 dense_3 (Dense)             (None, 12)                204       
                                                                 
 dense_4 (Dense)             (None, 8)                 104       
                                                                 
 dense_5 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,925
Trainable params: 1,925
Non-trainabl

#### Train the model
Each ``epoch`` is a complete pass through the entire training dataset, while the ``batch_size`` is the number of training samples the model processes before updating its weights

In [7]:
history_simple = model_simple.fit(model_dict['X_tr'], model_dict['y_tr'],
                                  epochs=10, batch_size=128,
                                  validation_data=(model_dict['X_val'], model_dict['y_val']))

Epoch 1/10
214/214 [==============================] - 1s 5ms/step - loss: 0.6627 - accuracy: 0.5071 - val_loss: 0.5943 - val_accuracy: 0.6692
Epoch 2/10
214/214 [==============================] - 1s 5ms/step - loss: 0.5265 - accuracy: 0.6724 - val_loss: 0.4673 - val_accuracy: 0.6795
Epoch 3/10
214/214 [==============================] - 1s 5ms/step - loss: 0.4406 - accuracy: 0.6723 - val_loss: 0.4089 - val_accuracy: 0.6795
Epoch 4/10
214/214 [==============================] - 1s 5ms/step - loss: 0.3992 - accuracy: 0.6752 - val_loss: 0.3790 - val_accuracy: 0.6810
Epoch 5/10
214/214 [==============================] - 1s 4ms/step - loss: 0.3680 - accuracy: 0.6871 - val_loss: 0.3572 - val_accuracy: 0.6890
Epoch 6/10
214/214 [==============================] - 1s 5ms/step - loss: 0.3402 - accuracy: 0.7088 - val_loss: 0.3398 - val_accuracy: 0.6939
Epoch 7/10
214/214 [==============================] - 1s 5ms/step - loss: 0.3135 - accuracy: 0.7321 - val_loss: 0.3271 - val_accuracy: 0.6917
Epoch 

#### Plot the training and validation loss

In [8]:
rst.plot_history(history_simple, "Simple")

#### Calculate accuracy and precision of the model

In [9]:
y_pred_simple = model_simple.predict(model_dict['X_ts'])
metrics_simple = rst.evaluate(y_pred_simple, model_dict['y_ts'], show_report = True)

169/169 [==============================] - 0s 1ms/step

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      3918
           1       0.83      0.77      0.80      1473

    accuracy                           0.89      5391
   macro avg       0.87      0.86      0.86      5391
weighted avg       0.89      0.89      0.89      5391

-----------------------------
Accuracy: 0.8932
Precision: 0.8252
Recall: 0.7726
F1_Score: 0.7980
MSE: 0.0621
MAE: 0.1530
R2: 0.6543
-----------------------------



#### And finally, save the model

In [10]:
model_name = f'{time.date()}_simple'
rst.save_model(modeling, model_simple, model_name)

Model '2025-06-12_simple' saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_simple.keras


---
#### 5. Now that we have a simple model trained, we can start building more complex models

To make our artificial networks as real as possible, we can let them see a sequence of frames to decide if the mouse is exploring
- Our build_RNN function will use Bidirectional LSTM layers that allow the model to take into account the temporal sequence of frames
- It also implements early stopping and learning rate scheduler mechanisms that will prevent the model from overfitting

We can control the RNN model by changing the following parameters on the modeling.yaml file:

`units` : The number of neurons on each layer of the LSTM model

`batch_size` : The number of training samples the model processes before updating its weights

`lr` : The learning rate of the model

`epochs` : The number of times the model will be trained on the entire training dataset

`past` & `future` : If you use a LSTM model, you can set the window size by saying how many frames into the past and how many into the future you want to see.

`broad` : Once you have your window size, we can broaden the window by skipping some frames as it strays further from the present.

In [11]:
model_wide = rst.build_RNN(modeling, model_dict)

Model: "CleanedBidirectionalRNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_sequence (InputLayer)  [(None, 5, 12)]          0         
                                                                 
 bidirectional (Bidirectiona  (None, 5, 64)            11520     
 l)                                                              
                                                                 
 bn_0 (BatchNormalization)   (None, 5, 64)             256       
                                                                 
 dropout_0 (Dropout)         (None, 5, 64)             0         
                                                                 
 bidirectional_1 (Bidirectio  (None, 5, 32)            10368     
 nal)                                                            
                                                                 
 bn_1 (BatchNormalization)   (None, 5, 32) 

#### Train the model

In [12]:
model_wide_name = f'{time.date()}_wide'
history_wide = rst.train_RNN(modeling, model_wide, model_dict, model_wide_name)


Epoch 1: LearningRateScheduler setting learning rate to 1.161875889658824e-05.
Epoch 1/100
27/27 [==============================] - 8s 96ms/step - loss: 1.2885 - accuracy: 0.2271 - val_loss: 0.7145 - val_accuracy: 0.2724 - lr: 1.1619e-05

Epoch 2: LearningRateScheduler setting learning rate to 1.7485542684453015e-05.
Epoch 2/100
27/27 [==============================] - 1s 38ms/step - loss: 1.2545 - accuracy: 0.2327 - val_loss: 0.7152 - val_accuracy: 0.3101 - lr: 1.7486e-05

Epoch 3: LearningRateScheduler setting learning rate to 3.790229669851489e-05.
Epoch 3/100
27/27 [==============================] - 1s 39ms/step - loss: 1.1886 - accuracy: 0.2502 - val_loss: 0.7118 - val_accuracy: 0.3456 - lr: 3.7902e-05

Epoch 4: LearningRateScheduler setting learning rate to 7.209770330148512e-05.
Epoch 4/100
27/27 [==============================] - 1s 38ms/step - loss: 1.0831 - accuracy: 0.2802 - val_loss: 0.6964 - val_accuracy: 0.3829 - lr: 7.2098e-05

Epoch 5: LearningRateScheduler setting lea

#### Plot the training and validation loss

In [13]:
rst.plot_history(history_wide, model_wide_name)
rst.plot_lr_schedule(history_wide)

#### Calculate accuracy and precision of the model

In [14]:
y_pred_wide = model_wide.predict(model_dict['X_ts_wide'])
metrics_wide = rst.evaluate(y_pred_wide, model_dict['y_ts'], show_report=True)

169/169 [==============================] - 3s 8ms/step

--- Classification Report ---
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      3918
           1       0.76      0.92      0.84      1473

    accuracy                           0.90      5391
   macro avg       0.87      0.91      0.88      5391
weighted avg       0.91      0.90      0.90      5391

-----------------------------
Accuracy: 0.9009
Precision: 0.7639
Recall: 0.9226
F1_Score: 0.8358
MSE: 0.0669
MAE: 0.1950
R2: 0.6277
-----------------------------



#### Save the wide model

In [15]:
rst.save_model(modeling, model_wide, model_wide_name)

Model '2025-06-12_wide' saved to: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_wide.keras


---
#### 6. Finally, compare the trained models

- Since we trained using the training dataset, and validated using the validation dataset... we test each model using the testing dataset.

In [16]:
# Print the model results
print("Evaluate model vs testing data")
print(f"{metrics_simple} -> simple")
print(f"{metrics_wide} -> wide")

Evaluate model vs testing data
{'Accuracy': 0.8931552587646077, 'Precision': 0.8252356780275562, 'Recall': 0.7725729803122878, 'F1_Score': 0.7980364656381487, 'MSE': 0.06207480504933027, 'MAE': 0.153030686116294, 'R2': 0.6543119759365662} -> simple
{'Accuracy': 0.900946021146355, 'Precision': 0.7639123102866779, 'Recall': 0.9226069246435845, 'F1_Score': 0.8357933579335793, 'MSE': 0.06686195991336391, 'MAE': 0.1950133517386851, 'R2': 0.6276528168056058} -> wide


---
---
#### Our trained models are stored safely in our repository, with today's date.
We can:
- Continue on this notebook and evaluate the trained models.
- Skip the evaluation and go use the models on our data in `3b-Automatic_analysis`.
---


## Evaluate models

I see you've decided to continue on this notebook! You wont regret it.

One may think that the evaluation we did on the testing set is enough, and in many cases it is. However, for our purpose of finding a model that resembles the labeling of an expert, It's better to compare the performance of the model against all the manually labeled data we have.

In [17]:
evaluation_dict = rst.build_evaluation_dict(modeling)

---
#### 7. Calculate a good reference labeler
Since we want to compare the models and the labelers, we need to create a reference labeler.

This reference could be the mean of all the labelers, but then the labelers would have an unfair advantage.

To avoid this, we choose to simultaneously create a chimera labeler and a leave-one-out-mean:
- The chimera is created by randomly selecting a labeler on each row of the data.
- The leave-one-out-mean is created by averaging the remaining labelers.

This way, we can compare the chimera to the leave-one-out-mean knowing that they are independent.

In [18]:
chimera_dict = rst.create_chimera_and_loo_mean(evaluation_dict['manual_labels'], seed=42)

---
#### 8. Load the models & use them to label exploration on all the available data

In [19]:
# List the models you want to evaluate
model_paths = {
    'example_simple': models_folder / 'trained_models/example_simple.keras',
    'example_wide': models_folder / 'trained_models/example_wide.keras',
    f'{time.date()}_simple': models_folder / f'trained_models/{time.date()}_simple.keras',
    f'{time.date()}_wide': models_folder / f'trained_models/{time.date()}_wide.keras',
    # Add more models as needed...
    }

In [20]:
models_dict = rst.build_and_run_models(modeling, model_paths, evaluation_dict['position'])

Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_simple.keras
5220/5220 [==============================] - 6s 1ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_wide.keras
5220/5220 [==============================] - 44s 8ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_simple.keras
5220/5220 [==============================] - 7s 1ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_wide.keras
5220/5220 [==============================] - 37s 7ms/step


In [21]:
evaluation_dict.update(chimera_dict)
evaluation_dict.update(models_dict)
evaluation_dict = {k: v for k, v in evaluation_dict.items() if k not in {'position', 'manual_labels'}}
print(evaluation_dict.keys())  # Check the keys to confirm the additions

dict_keys(['Labeler_A', 'Labeler_B', 'Labeler_C', 'Labeler_D', 'Labeler_E', 'geometric', 'chimera', 'loo_mean', 'model_example_simple', 'model_example_wide', 'model_2025-06-12_simple', 'model_2025-06-12_wide'])


---
#### 9. With all the labels organized, we can evaulate the performance of each

In [22]:
for name, pred in evaluation_dict.items():
    metrics = rst.evaluate(pred, evaluation_dict['loo_mean'])
    print(f"{metrics} -> {name}")

{'Accuracy': 0.9769118386702752, 'Precision': 0.7069446597426755, 'Recall': 0.991844280121792, 'F1_Score': 0.825504570549371, 'MSE': 0.01142170921849927, 'MAE': 0.02253281201350801, 'R2': 0.7654496770367311} -> Labeler_A
{'Accuracy': 0.9825940650971188, 'Precision': 0.9736406085253804, 'Recall': 0.702914310569813, 'F1_Score': 0.8164193242816545, 'MSE': 0.015732776686705148, 'MAE': 0.02684387948171389, 'R2': 0.6769198215098198} -> Labeler_B
{'Accuracy': 0.9648408497592987, 'Precision': 0.6118890534536152, 'Recall': 0.98836450630709, 'F1_Score': 0.7558419958419959, 'MSE': 0.018322410964481593, 'MAE': 0.029433513759490337, 'R2': 0.623740429127335} -> Labeler_C
{'Accuracy': 0.9901504083538908, 'Precision': 0.8752609084584038, 'Recall': 0.9575902566333189, 'F1_Score': 0.9145765176299527, 'MSE': 0.008550658036548272, 'MAE': 0.019661760831557015, 'R2': 0.8244081016550004} -> Labeler_D
{'Accuracy': 0.9927729743970494, 'Precision': 0.9476633419253614, 'Recall': 0.9195302305350153, 'F1_Score': 0

We can visualize the similarity between labelers using a cosine similarity plot

In [23]:
rst.plot_cosine_sim(evaluation_dict)

And finally, run a PCA (Principal Components Analysis) to see how much each labeler resembles eachother and the mean

In [24]:
rst.plot_PCA(evaluation_dict, make_discrete=False)

Also, we can see both the models and the labelers performance on an example video

In [25]:
example_folder_path = base / r'examples\colabeled_video'

labelers_example = {
    "labeler_A": "Example_labeler_A.csv",
    "labeler_B": "Example_labeler_B.csv",
    "labeler_C": "Example_labeler_C.csv",
    "labeler_D": "Example_labeler_D.csv",
    "labeler_E": "Example_labeler_E.csv"
}

rst.plot_performance_on_video(example_folder_path, model_paths, labelers_example, fps = 25, 
                              bodyparts = ['nose', 'left_ear', 'right_ear', 'head', 'neck', 'body'], 
                              targets = ['obj_1', 'obj_2'], plot_tgt = "obj_2")

Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_simple.keras
469/469 [==============================] - 1s 1ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\example_wide.keras
469/469 [==============================] - 6s 9ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_simple.keras
469/469 [==============================] - 1s 1ms/step
Loading model from: c:\Users\dhers\Desktop\Rainstorm\examples\models\trained_models\2025-06-12_wide.keras
469/469 [==============================] - 5s 7ms/step


---
---
#### Once we get to this point, we should have selected our favorite model.
We can move on to the next notebook, `3b-Automatic_analysis`, and use the chosen model to label our position files.

---
RAINSTORM - Created on Dec 12, 2023 - @author: Santiago D'hers